In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


# Visualizing the data

In [2]:
train_df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [4]:
from bs4 import BeautifulSoup
!pip install contractions
import contractions
from textblob import TextBlob
import spacy 
import re
import string

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.5 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
def remove_tweet_username(df):
    return re.sub('@[^\s]+','', df)

def make_lower(df):
    return df.lower()

def cont_exp(df):
    return contractions.fix(df)

def make_string(df):
    return str(df)

def remove_url(df):
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , df)

def remove_email(df):
    return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", df)

#Retweets
def remove_rt(df):
    df = str(df)
    return re.sub(r'\brt\b', "", df).strip()

def remove_html(df):
    return BeautifulSoup(df, 'lxml').get_text().strip()
def remove_dots(df):
    dot_pattern = re.compile(r'\.{1,}')
    single_dot = dot_pattern.sub(' ', df)
    return single_dot

def remove_special_chars(df):
    df = re.sub(r'[^\w]+', " ", df)
    df = ' '.join(df.split())
    
    return df

def make_base(df):
    df = str(df)
    x_list = []
    doc = nlp(df)

    for token in doc:
        lemma = token.lemma_
        if lemma == '-PRON-' or lemma == 'be':
            lemma = token.text

        x_list.append(lemma)
    return ' '.join(x_list)

def spelling_correction(df):
    df = TextBlob(df).correct()
    return df
def resub(df):
    return re.sub("(.)\\1{2,}", "\\1", df)

def get_clean_data(df):
    df = remove_url(df)
    df = remove_email(df)
#    df = remove_special_chars(df)
    df = remove_html(df)
#    df = remove_dots(df)
#    df = make_base(df)
#    df = spelling_correction(df).raw_sentences[0]
    df = make_lower(df)
    df = make_string(df)
    df = cont_exp(df)
    df = remove_rt(df)
    df = resub(df)                             
    df = remove_tweet_username(df)
    
    return df

In [6]:
train_df['text'] = train_df['text'].apply(get_clean_data)

/tmp/ipykernel_28/4114496449.py:25: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(df, 'lxml').get_text().strip()


In [27]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                            train_df["target"].to_numpy(),
                                                                            test_size=0.1) # dedicate 10% of samples to validation set 

In [67]:
train=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

from sklearn.model_selection import train_test_split

trainX, valX, trainY, valY = train_test_split(train["text"].to_numpy(),
                                                                            train["target"].to_numpy(),
                                                                            test_size=0.1) # dedicate 10% of samples to validation set

In [9]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

# now we have cleaned data, time for building models


# 1. model RNN, LSTM, GRU, Bidirectional

In [10]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

14

In [11]:
# creating embeddings from tokens
from tensorflow.keras.layers import TextVectorization

In [12]:
# a text vectorisation layer on top of the model
max_length=14
max_vocab_length=10000
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                 output_mode='int',
                                 output_sequence_length=max_length)

In [29]:
# fit our data to vectorizer
text_vectorizer.adapt(train_sentences)

In [14]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'to']
Bottom 5 least common words: ['manly', 'manitou', 'manifesting', 'manifestation', 'manhood']


In [15]:
# now we have tokens for each text sentence, now time from embeddings
from keras.layers import Embedding

embedding_layer=Embedding(input_dim=max_vocab_length,
                         output_dim=128,
                         embeddings_initializer='uniform',
                         input_length=max_length,
                         name='Embedding')
#each sentence is of length 14 and each word is represnted using learnable 128 size vector

In [16]:
import random
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding_layer(text_vectorizer([random_sentence]))
sample_embed

Original text:
a friend is like blood they are not beside us always. but they come out when we are wounded.      

Embedded version:


<tf.Tensor: shape=(1, 14, 128), dtype=float32, numpy=
array([[[ 0.04652433,  0.02060845, -0.04968445, ..., -0.03054878,
         -0.04232873, -0.00330784],
        [-0.04634892,  0.00537262, -0.03423559, ...,  0.04979067,
          0.02523496, -0.04066994],
        [-0.03076838,  0.01311561,  0.03551148, ...,  0.04038313,
         -0.02300445,  0.00344958],
        ...,
        [ 0.04287304,  0.02265931, -0.01749256, ..., -0.04953734,
          0.0374471 ,  0.04841968],
        [-0.01695748, -0.02284133, -0.02001741, ..., -0.04414875,
         -0.03329112, -0.04970726],
        [-0.01808946, -0.01030345,  0.0076587 , ...,  0.0194551 ,
         -0.00089896,  0.02501665]]], dtype=float32)>

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [18]:
model0_preds = model_0.predict(val_sentences)


In [19]:
model_0.score(val_sentences,val_labels)

0.7847769028871391

In [20]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [21]:
naive_model_result=calculate_results(y_true=val_labels,
                                    y_pred=model0_preds)
naive_model_result

{'accuracy': 78.4776902887139,
 'precision': 0.7990968040759612,
 'recall': 0.7847769028871391,
 'f1': 0.7775051802735186}

# using GRU and LSTM 

In [64]:
import tensorflow as tf
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype='string') #inputs are 1D text strings
x=text_vectorizer(inputs) # convert inputs to numbers
x=embedding_layer(x) # convert numbers to vectors (output=(1,15,128))
x=layers.GlobalAveragePooling1D()(x) # output (1,128)
# x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name='model_1')

In [65]:
model_1.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam())

In [34]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 14)               0         
 torization)                                                     
                                                                 
 Embedding (Embedding)       (None, 14, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-tra

In [66]:
model_1.fit(train_sentences,
           train_labels,
            epochs=5,
            validation_data=(val_sentences,val_labels),
           )

Epoch 1/5
215/215 [==============================] - 10s 42ms/step - loss: 0.4506 - val_loss: 0.5110
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2290 - val_loss: 0.5312
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1610 - val_loss: 0.5748
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1268 - val_loss: 0.6254
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1063 - val_loss: 0.6740


In [68]:
ml=15
max_vocab_length=10000
vec=TextVectorization(max_tokens=max_vocab_length,
                                 output_mode='int',
                                 output_sequence_length=ml)
vec.adapt(trainX)

emb=Embedding(input_dim=max_vocab_length,
                         output_dim=128,
                         embeddings_initializer='uniform',
                         input_length=ml,
                         name='Embedding')
inputs=layers.Input(shape=(1,),dtype='string') #inputs are 1D text strings
x=vec(inputs) # convert inputs to numbers
x=emb(x) # convert numbers to vectors (output=(1,15,128))
x=layers.GlobalAveragePooling1D()(x) # output (1,128)
# x=layers.Dense(1024,activation='relu')(x)
# x=layers.Dense(256,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1_dummy=tf.keras.Model(inputs,outputs,name='model_1')

model_1_dummy.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam())

model_1_dummy.fit(trainX,
           trainY,
            epochs=5,
            validation_data=(valX,valY),
           )
        

Epoch 1/5
215/215 [==============================] - 11s 45ms/step - loss: 0.6147 - val_loss: 0.5358
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4445 - val_loss: 0.4645
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3497 - val_loss: 0.4366
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2866 - val_loss: 0.4417
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2389 - val_loss: 0.4585


In [52]:
# cleaned data
model_1_preds_prob=model_1.predict(val_sentences)
model_1_preds = tf.squeeze(tf.round(model_1_preds_prob))
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_1_preds)
model_1_results

24/24 [==============================] - 0s 2ms/step


{'accuracy': 75.8530183727034,
 'precision': 0.7570566171841299,
 'recall': 0.7585301837270341,
 'f1': 0.756583348908012}

In [53]:
# raw data
model_1_preds_prob=model_1_dummy.predict(valX)
model_1_preds = tf.squeeze(tf.round(model_1_preds_prob))
model_1_results = calculate_results(y_true=valY, 
                                    y_pred=model_1_preds)
model_1_results

24/24 [==============================] - 0s 2ms/step


{'accuracy': 80.70866141732283,
 'precision': 0.8072706481168199,
 'recall': 0.8070866141732284,
 'f1': 0.8061454577074249}

# pre-trained models

In [69]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder

In [70]:
# this model converts each sentence into vector of 512 dimensions, it does not depend on the size of text

# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") 

In [81]:
# cleaned data
pretrained_model=tf.keras.Sequential()
pretrained_model.add(sentence_encoder_layer)
# pretrained_model.add(layers.Dense(256,activation='relu'))
# pretrained_model.add(layers.Dense(64,activation='relu'))
pretrained_model.add(layers.Dense(32,activation='relu'))
pretrained_model.add(layers.Dense(1,activation='sigmoid'))


pretrained_model.compile(loss='binary_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy'])
pretrained_model.fit(train_sentences,
            train_labels,
            epochs=8,
            validation_data=(val_sentences, val_labels))

Epoch 1/8
215/215 [==============================] - 4s 14ms/step - loss: 0.5336 - accuracy: 0.7749 - val_loss: 0.4631 - val_accuracy: 0.7677
Epoch 2/8
215/215 [==============================] - 3s 12ms/step - loss: 0.4193 - accuracy: 0.8135 - val_loss: 0.4479 - val_accuracy: 0.7953
Epoch 3/8
215/215 [==============================] - 3s 12ms/step - loss: 0.4014 - accuracy: 0.8178 - val_loss: 0.4534 - val_accuracy: 0.7953
Epoch 4/8
215/215 [==============================] - 3s 12ms/step - loss: 0.3950 - accuracy: 0.8235 - val_loss: 0.4471 - val_accuracy: 0.7992
Epoch 5/8
215/215 [==============================] - 3s 12ms/step - loss: 0.3882 - accuracy: 0.8263 - val_loss: 0.4480 - val_accuracy: 0.8110
Epoch 6/8
215/215 [==============================] - 3s 12ms/step - loss: 0.3839 - accuracy: 0.8308 - val_loss: 0.4473 - val_accuracy: 0.8071
Epoch 7/8
215/215 [==============================] - 3s 13ms/step - loss: 0.3797 - accuracy: 0.8320 - val_loss: 0.4483 - val_accuracy: 0.8045
Epoch 

In [84]:
# raw data
pretrained_model=tf.keras.Sequential()
pretrained_model.add(sentence_encoder_layer)
# pretrained_model.add(layers.Dense(256,activation='relu'))
pretrained_model.add(layers.Dense(64,activation='relu'))
pretrained_model.add(layers.Dropout(0.3))
pretrained_model.add(layers.Dense(32,activation='relu'))
pretrained_model.add(layers.Dense(1,activation='sigmoid'))


pretrained_model.compile(loss='binary_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy'])
pretrained_model.fit(trainX,
            trainY,
            epochs=5,
            validation_data=(valX, valY))


Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.5133 - accuracy: 0.7689 - val_loss: 0.4084 - val_accuracy: 0.8268
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4146 - accuracy: 0.8196 - val_loss: 0.4020 - val_accuracy: 0.8241
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3953 - accuracy: 0.8273 - val_loss: 0.3985 - val_accuracy: 0.8241
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3771 - accuracy: 0.8409 - val_loss: 0.4058 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3624 - accuracy: 0.8485 - val_loss: 0.4029 - val_accuracy: 0.8189


In [116]:
model_6_preds=tf.squeeze(tf.round(pretrained_model.predict(valX)))
model_6_results = calculate_results(valY, model_6_preds)

model_6_preds.shape

24/24 [==============================] - 1s 15ms/step


TensorShape([762])

# last model is giving the best result so far, so now we will train it on whole data, and predict on the test data

In [85]:
data=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

train_X=data['text']
train_Y=data['target']

pretrained_model=tf.keras.Sequential()
pretrained_model.add(sentence_encoder_layer)
# pretrained_model.add(layers.Dense(256,activation='relu'))
pretrained_model.add(layers.Dense(64,activation='relu'))
pretrained_model.add(layers.Dropout(0.3))
pretrained_model.add(layers.Dense(32,activation='relu'))
pretrained_model.add(layers.Dense(1,activation='sigmoid'))


pretrained_model.compile(loss='binary_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy'])
pretrained_model.fit(train_X,
            train_Y,
            epochs=5)


Epoch 1/5
238/238 [==============================] - 5s 13ms/step - loss: 0.4996 - accuracy: 0.7739
Epoch 2/5
238/238 [==============================] - 3s 12ms/step - loss: 0.4134 - accuracy: 0.8221
Epoch 3/5
238/238 [==============================] - 3s 11ms/step - loss: 0.3943 - accuracy: 0.8325
Epoch 4/5
238/238 [==============================] - 3s 11ms/step - loss: 0.3812 - accuracy: 0.8358
Epoch 5/5
238/238 [==============================] - 3s 11ms/step - loss: 0.3645 - accuracy: 0.8468


In [115]:
test_data=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_data.head()
test_data.shape

(3263, 4)

In [112]:
test_X=test_data['text']
pred_probablties=pretrained_model.predict(test_X)
pred_probablties.shape


102/102 [==============================] - 1s 10ms/step


(3263, 1)

In [117]:
prediction=tf.squeeze(tf.round(pretrained_model.predict(test_X)))

102/102 [==============================] - 1s 9ms/step


In [118]:
test_id=test_data['id']
test_id.shape

(3263,)

In [125]:
output=pd.DataFrame({'id':test_id, 'target':prediction})
output['target']=output['target'].astype(int)

In [126]:
output.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [127]:
output.to_csv('submission.csv',index=False)